This code defines a PyTorch **Function** called **ReverseLayerF**. A PyTorch Function is a way to define custom operations for the PyTorch autograd engine to perform backpropagation on.

The **forward** method defines the computation performed during the forward pass of the network. It takes in two inputs, **x** and **alpha**, and assigns the value of **alpha** to a context variable **ctx.alpha**. The context variable is used to store information that needs to be reused during the backward pass. The method then returns **x** unmodified.

The **backward** method defines the computation performed during the backward pass of the network. It takes in one input, **grad_output**, which is the gradient of the output with respect to the input. It negates this gradient and multiplies it by **ctx.alpha**, which was stored during the forward pass. The method then returns the resulting gradient, and a **None** value, as the second input does not require gradient computation.

This **ReverseLayerF** is used in the architecture of domain adaptation model (DANN) which is a technique that enables to train a model on one domain and apply it to another domain. This function uses the gradient reversal layer to reverse the gradients during the backpropagation process.

In [ ]:
from torch.autograd import Function


class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

This code defines a PyTorch **nn.Module** called **CNNModel** which is a CNN model for domain adaptation. The model has two classifiers: a classifier for the original task, and a domain classifier that aims to predict the domain of the input image. The model also applies a reverse layer feature on the input image to train the domain classifier.

In the constructor, the class initializes several layers of the model. The first part is the feature extractor, which consists of a series of convolutional layers, batch normalization layers, max pooling layers, and ReLU activation functions. This part of the model is used to extract features from the input image.

The second part is the class classifier, which takes the features extracted by the feature extractor and processes them through a series of linear layers, batch normalization layers, dropout layers and ReLU activation functions. The output of this classifier is the predicted class label.

The third part is the domain classifier, which also takes the features extracted by the feature extractor and processes them through a series of linear layers, batch normalization layers, dropout layers, and ReLU activation functions. The output of this classifier is the predicted domain label.

The **forward** method of the class defines the forward pass of the model. It takes in two inputs, **input_data** and **alpha**. The input_data is expanded to a shape of (batch_size, 3, 28, 28) and passed through the feature extractor. The features are then reshaped to (batch_size, 50 * 4 * 4) and passed through the ReverseLayerF function with the alpha value as input. The output of the ReverseLayerF function is then passed through the domain classifier, and the original feature is passed through the class classifier. The method returns the output of both classifiers.

In [ ]:
import torch.nn as nn


class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel, self).__init__()
        self.feature = nn.Sequential()
        self.feature.add_module('f_conv1', nn.Conv2d(3, 64, kernel_size=5))
        self.feature.add_module('f_bn1', nn.BatchNorm2d(64))
        self.feature.add_module('f_pool1', nn.MaxPool2d(2))
        self.feature.add_module('f_relu1', nn.ReLU(True))
        self.feature.add_module('f_conv2', nn.Conv2d(64, 50, kernel_size=5))
        self.feature.add_module('f_bn2', nn.BatchNorm2d(50))
        self.feature.add_module('f_drop1', nn.Dropout2d())
        self.feature.add_module('f_pool2', nn.MaxPool2d(2))
        self.feature.add_module('f_relu2', nn.ReLU(True))

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(50 * 4 * 4, 100))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout2d())
        self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(100, 10))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax())

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(50 * 4 * 4, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, 28, 28)
        feature = self.feature(input_data)
        feature = feature.view(-1, 50 * 4 * 4)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

This code defines a custom PyTorch **Dataset** class called **GetLoader**. This class is used to load and preprocess data for a deep learning model. The class is initialized with three inputs, **data_root**, **data_list**, and **transform**.

**data_root** is the root directory where the images are stored. **data_list** is the path to a text file containing the list of image file names and labels, with each line in the format "path/to/image label". **transform** is an optional argument for a PyTorch **transforms **object, which is used to apply data preprocessing operations, such as image resizing, normalization, etc.

The **__init__** method reads the file specified in **data_list** and separates the image paths and labels into two lists, **self.img_paths** and **self.img_labels**. **self.n_data** stores the total number of images in the dataset.

The **__getitem__** method is used to retrieve a single sample from the dataset. It takes in one input, **item**, which is an index of the sample. The method retrieves the image path and label corresponding to the index and opens the image using the PIL library. If a **transform** is specified, the image is passed through the **transform** before returning the image and label.

The **__len__** method returns the total number of samples in the dataset. This method is needed by PyTorch DataLoader to know the total number of samples.

This class can be used to load and preprocess image datasets for training and testing a deep learning model. It allows for easy access to the dataset and applies the specified data preprocessing operations on the images.

In [ ]:
import torch.utils.data as data
from PIL import Image
import os


class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list, transform=None):
        self.root = data_root
        self.transform = transform

        f = open(data_list, 'r')
        data_list = f.readlines()
        f.close()

        self.n_data = len(data_list)

        self.img_paths = []
        self.img_labels = []

        for data in data_list:
            self.img_paths.append(data[:-3])
            self.img_labels.append(data[-2])

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item], self.img_labels[item]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels

    def __len__(self):
        return self.n_data

This code is a function called **test** that tests a trained deep learning model on a given dataset. The function takes in two inputs, **dataset_name** and epoch, which specify the dataset and the **epoch** of the trained model to be tested, respectively.

The function starts by checking that the provided **dataset_name** is either "MNIST" or "mnist_m", and sets the paths to the model and image roots. It then sets some hyperparameters such as **cuda**, **cudnn.benchmark**, **batch_size**, **image_size**, and **alpha**.

Then it loads the data by creating a **transforms.Compose** object that includes resizing the images to 28x28 pixels, converting them to tensors and normalizing them. Then it creates a **DataLoader** object that loads the data in batches and shuffles it.

It then loads the trained model by its name and the specified** epoch**, sets it to evaluation mode and sends it to the GPU if cuda is true.

It then iterates over the dataloader, and for each batch, it tests the model using target data, it gets the predicted class and compares it to the real class, and if they match, it increments the n_correct counter, and in the end, it calculates the accuracy by dividing n_correct by n_total and prints the accuracy of the dataset.

The function is used for testing a pre-trained model on a dataset and calculates the accuracy of the model on the dataset.

In [ ]:
import torch.backends.cudnn as cudnn
import torch.utils.data
from torchvision import transforms
from torchvision import datasets


def test(dataset_name, epoch):
    assert dataset_name in ['MNIST', 'mnist_m']

    model_root = os.path.join('..', 'models')
    image_root = os.path.join('..', 'dataset', dataset_name)

    cuda = True
    cudnn.benchmark = True
    batch_size = 128
    image_size = 28
    alpha = 0

    """load data"""

    img_transform_source = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.1307,), std=(0.3081,))
    ])

    img_transform_target = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])

    if dataset_name == 'mnist_m':
        test_list = os.path.join(image_root, 'mnist_m_test_labels.txt')

        dataset = GetLoader(
            data_root=os.path.join(image_root, 'mnist_m_test'),
            data_list=test_list,
            transform=img_transform_target
        )
    else:
        dataset = datasets.MNIST(
            root='../dataset',
            train=False,
            transform=img_transform_source,
        )

    dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8
    )

    """ training """

    my_net = torch.load(os.path.join(
        model_root, 'mnist_mnistm_model_epoch_' + str(epoch) + '.pth'
    ))
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = next(data_target_iter)
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)

        class_output, _ = my_net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(class_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct.data.numpy() * 1.0 / n_total

    print('epoch: %d, accuracy of the %s dataset: %f' % (epoch, dataset_name, accu))

The **mkdir** command is a command-line utility that creates a new directory with the specified name. In this case, it creates a new directory called "dataset". If the directory already exists, it will return an error.

In [ ]:
!mkdir dataset

Doing the same to create "models" directory.

In [ ]:
!mkdir models

This code is a command-line command that changes the current working directory to the "dataset" directory.

In [ ]:
cd dataset

/content/dataset


The **pip install** command is used to install Python packages. The **--upgrade** flag is used to upgrade an existing package to the latest version. **gdown** is a package that allows you to download large files from Google Drive using the command line. It is useful when working with large datasets that are hosted on Google Drive and need to be downloaded to the local machine for processing. 

In [ ]:
!pip install --upgrade gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


The **gdown.download()** function takes in the Google Drive shareable link and the filename of the file that we want to download. This code uses the **gdown** package to download the file 'mnist_m.tar.gz' from the Google Drive URL, and saves it in the current working directory. The quiet parameter is set to False, so the download progress will be displayed in the console.

In [ ]:
import gdown
gdown.download('https://drive.google.com/u/0/uc?id=0B_tExHiYS-0veklUZHFYT19KYjg&export=download', 'mnist_m.tar.gz', quiet=False)

Downloading...
From: https://drive.google.com/u/0/uc?id=0B_tExHiYS-0veklUZHFYT19KYjg&export=download
To: /content/dataset/mnist_m.tar.gz
100%|██████████| 134M/134M [00:02<00:00, 50.4MB/s]


'mnist_m.tar.gz'

In this command, the file mnist_m.tar.gz is decompressed and its content is extracted in the current working directory. The **tar** command takes the archive file 'mnist_m.tar.gz', decompresses it and extracts the contents of the archive. The **-z** flag is used to indicate that the archive is compressed using gzip and the **-x** flag is used to extract the contents of the archive. The **-v** flag is used to display the progress of the extraction process in the console. The **-f** option specifies the archive file name.

In [ ]:
!tar -zvxf mnist_m.tar.gz

Streaming output truncated to the last 5000 lines.
mnist_m/mnist_m_test/00008464.png
mnist_m/mnist_m_test/00004544.png
mnist_m/mnist_m_test/00000258.png
mnist_m/mnist_m_test/00004074.png
mnist_m/mnist_m_test/00006977.png
mnist_m/mnist_m_test/00004200.png
mnist_m/mnist_m_test/00005651.png
mnist_m/mnist_m_test/00008563.png
mnist_m/mnist_m_test/00008032.png
mnist_m/mnist_m_test/00003311.png
mnist_m/mnist_m_test/00003541.png
mnist_m/mnist_m_test/00003759.png
mnist_m/mnist_m_test/00005223.png
mnist_m/mnist_m_test/00008205.png
mnist_m/mnist_m_test/00007685.png
mnist_m/mnist_m_test/00005618.png
mnist_m/mnist_m_test/00001756.png
mnist_m/mnist_m_test/00005528.png
mnist_m/mnist_m_test/00008606.png
mnist_m/mnist_m_test/00005159.png
mnist_m/mnist_m_test/00005128.png
mnist_m/mnist_m_test/00001825.png
mnist_m/mnist_m_test/00001524.png
mnist_m/mnist_m_test/00003892.png
mnist_m/mnist_m_test/00006887.png
mnist_m/mnist_m_test/00007919.png
mnist_m/mnist_m_test/00005374.png
mnist_m/mnist_m_test/00000191.p

This script is the PyTorch implementation of our Deep Adaptation Neural Network. The script first loads the MNIST and MNIST-M datasets and applies data augmentation to them. Then, it creates an instance of the CNNModel class, which is a neural network with two branches: one for classifying the images and another for classifying the domain of the images. The script then sets up an optimizer (Adam) and two loss functions (cross-entropy loss for classification and domain classification).

The script then enters the main training loop, which runs for n_epoch number of times. In each epoch, the script iterates over both the source and target dataloaders and performs the following steps:

It sets the alpha value, which controls the weighting of the domain classification loss
It loads the next batch of data from the source and target dataloaders
It sets the gradients to zero
It performs a forward pass of the data through the network, and computes the classification and domain classification losses
It computes the gradients with respect to the parameters of the network
It updates the parameters of the network using the optimizer
After training, the script saves the trained model and prints the accuracy of the model on the target dataset.

In [ ]:
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import numpy as np

source_dataset_name = 'MNIST'
target_dataset_name = 'mnist_m'
source_image_root = os.path.join('..', 'dataset', source_dataset_name)
target_image_root = os.path.join('..', 'dataset', target_dataset_name)
model_root = os.path.join('..', 'models')
cuda = True
cudnn.benchmark = True
lr = 1e-3
batch_size = 128
image_size = 28
n_epoch = 20

manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# load data

img_transform_source = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

img_transform_target = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

dataset_source = datasets.MNIST(
    root='../dataset',
    train=True,
    transform=img_transform_source,
    download=True
)

dataloader_source = torch.utils.data.DataLoader(
    dataset=dataset_source,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)

train_list = os.path.join(target_image_root, 'mnist_m_train_labels.txt')

dataset_target = GetLoader(
    data_root=os.path.join(target_image_root, 'mnist_m_train'),
    data_list=train_list,
    transform=img_transform_target
)

dataloader_target = torch.utils.data.DataLoader(
    dataset=dataset_target,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8)

# load model

my_net = CNNModel()

# setup optimizer

optimizer = optim.Adam(my_net.parameters(), lr=lr)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

for p in my_net.parameters():
    p.requires_grad = True

# training

for epoch in range(n_epoch):

    len_dataloader = min(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = next(data_source_iter)
        s_img, s_label = data_source

        my_net.zero_grad()
        batch_size = len(s_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)

        class_output, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        data_target = next(data_target_iter)
        t_img, _ = data_target

        batch_size = len(t_img)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)

        _, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        i += 1

        print('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' %
              (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
              err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

    torch.save(my_net, '{0}/mnist_mnistm_model_epoch_{1}.pth'.format(model_root, epoch))
    test(source_dataset_name, epoch)
    test(target_dataset_name, epoch)

print ('done')

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../dataset/MNIST/raw/train-images-idx3-ubyte.gz to ../dataset/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/MNIST/raw



/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
/usr/local/lib/python3.8/dist-packages/torch/nn/module

Streaming output truncated to the last 5000 lines.
epoch: 9, [iter: 95 / all 461], err_s_label: 0.172798, err_s_domain: 0.645555, err_t_domain: 0.686889
epoch: 9, [iter: 96 / all 461], err_s_label: 0.191312, err_s_domain: 0.686684, err_t_domain: 0.648045
epoch: 9, [iter: 97 / all 461], err_s_label: 0.121572, err_s_domain: 0.651685, err_t_domain: 0.655653
epoch: 9, [iter: 98 / all 461], err_s_label: 0.157450, err_s_domain: 0.684454, err_t_domain: 0.660367
epoch: 9, [iter: 99 / all 461], err_s_label: 0.280150, err_s_domain: 0.619713, err_t_domain: 0.636594
epoch: 9, [iter: 100 / all 461], err_s_label: 0.271116, err_s_domain: 0.591704, err_t_domain: 0.643724
epoch: 9, [iter: 101 / all 461], err_s_label: 0.179688, err_s_domain: 0.614296, err_t_domain: 0.630117
epoch: 9, [iter: 102 / all 461], err_s_label: 0.214914, err_s_domain: 0.639275, err_t_domain: 0.641557
epoch: 9, [iter: 103 / all 461], err_s_label: 0.098383, err_s_domain: 0.621287, err_t_domain: 0.651200
epoch: 9, [iter: 104 / all 